In [1]:
from google import genai
from google.genai import types
import time


In [2]:
client = genai.Client()

In [11]:
# Upload the file using the Files API, supply a file name which will be visible in citations
sample_file = client.files.upload(file='microprocessorbook.pdf', config={'name': 'microprocessorbook1123'})

ClientError: 409 ALREADY_EXISTS. {'error': {'code': 409, 'message': 'microprocessorbook1123 already exists.', 'status': 'ALREADY_EXISTS'}}

In [12]:

# Create the File Search store with an optional display name
file_search_store = client.file_search_stores.create(config={'display_name': 'book2'})

In [13]:
#upload the file into the File Search store with a custom chunking configuration
operation = client.file_search_stores.import_file(
    file_search_store_name=file_search_store.name,
    file_name=sample_file.name,
    config={
        'chunking_config': {
          'white_space_config': {
            'max_tokens_per_chunk': 200,
            'max_overlap_tokens': 20
          }
        }
    }
)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You have exhausted your storage limit of 1000000000 bytes, please upgrade your quota tier to increase your storage limit. Please refer to https://ai.google.dev/gemini-api/docs/file-search#rate-limits for more information on limits.', 'status': 'RESOURCE_EXHAUSTED'}}

In [ ]:
# Wait until import is complete
while not operation.done:
    time.sleep(5)
    operation = client.operations.get(operation)

In [ ]:
# Ask a question about the file
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="""Cwhats the book is about""",
    config=types.GenerateContentConfig(
        tools=[
            types.Tool(
                file_search=types.FileSearch(
                    file_search_store_names=[file_search_store.name]
                )
            )
        ]
    )
)

In [7]:
## List all files uploaded to the general File API
print('My uploaded files:')
for file in client.files.list():
    print(f"Name: {file.name}, Display Name: {file.display_name}, Size: {file.size_bytes} bytes")

My uploaded files:
Name: files/microprocessorbook1123, Display Name: None, Size: 67871357 bytes
Name: files/microprocessorchunkingdemo, Display Name: None, Size: 67871357 bytes
Name: files/microprocessorbookmetadatademo, Display Name: None, Size: 67871357 bytes
Name: files/microprocessorbook113, Display Name: None, Size: 67871357 bytes
Name: files/microprocessorbook112, Display Name: None, Size: 67871357 bytes


In [8]:
# List all your File Search stores
for file_search_store in client.file_search_stores.list():
    print(file_search_store)

name='fileSearchStores/book2-878tfl7y8u29' display_name='book2' create_time=datetime.datetime(2025, 11, 28, 4, 56, 36, 814228, tzinfo=TzInfo(0)) update_time=datetime.datetime(2025, 11, 28, 4, 56, 36, 814228, tzinfo=TzInfo(0)) active_documents_count=1 pending_documents_count=None failed_documents_count=None size_bytes=67871357
name='fileSearchStores/book2-p9u7kahoazhk' display_name='book2' create_time=datetime.datetime(2025, 11, 28, 5, 5, 28, 534224, tzinfo=TzInfo(0)) update_time=datetime.datetime(2025, 11, 28, 5, 5, 28, 534224, tzinfo=TzInfo(0)) active_documents_count=2 pending_documents_count=None failed_documents_count=None size_bytes=135742714
name='fileSearchStores/metadatademostore-d8kpqug2arfz' display_name='metadatademostore' create_time=datetime.datetime(2025, 11, 28, 5, 29, 11, 778703, tzinfo=TzInfo(0)) update_time=datetime.datetime(2025, 11, 28, 5, 29, 11, 778703, tzinfo=TzInfo(0)) active_documents_count=1 pending_documents_count=None failed_documents_count=None size_bytes=67